# DeepForest: Análisis completo
Detección de árboles, análisis de índices de vegetación e interpretación RGB.

In [ ]:
# 1. Carga de librerías
from deepforest import main
from deepforest import get_data
from deepforest.visualize import plot_results
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# 2. Inicialización del modelo
model = main.deepforest()
model.load_model(model_name="weecology/deepforest-tree", revision="main")

In [ ]:
# 3. Carga de imagen y predicción
sample_image_path = "mi_imagen/imagen_taller7.jpg"
img_in = Image.open(sample_image_path)
display(img_in)
img_array = np.array(img_in)
img_out = model.predict_image(path=sample_image_path)
plot_results(img_out)
print(f"Número de árboles detectados: {len(img_out)}")

In [ ]:
# 4. Cálculo de áreas e ID
img_out["id_arbol"] = range(1, len(img_out) + 1)
img_out["area_px"] = (img_out["xmax"] - img_out["xmin"]) * (img_out["ymax"] - img_out["ymin"])

In [ ]:
# 5. Índices de vegetación
img_float = img_array.astype(float)
R = img_float[:, :, 0]
G = img_float[:, :, 1]
B = img_float[:, :, 2]
ExG = 2 * G - R - B
epsilon = 1e-6
VARI = (G - R) / (G + R - B + epsilon)
ExG_means = []
VARI_means = []
for _, row in img_out.iterrows():
    xmin, ymin, xmax, ymax = map(int, [row["xmin"], row["ymin"], row["xmax"], row["ymax"]])
    ExG_crop = ExG[ymin:ymax, xmin:xmax]
    VARI_crop = VARI[ymin:ymax, xmin:xmax]
    ExG_means.append(np.mean(ExG_crop))
    VARI_means.append(np.mean(VARI_crop))
img_out["ExG_mean"] = ExG_means
img_out["VARI_mean"] = VARI_means

In [ ]:
# 6. Mostrar cajas e IDs
fig, ax = plt.subplots(figsize=(12, 12))
ax.imshow(img_array)
for _, row in img_out.iterrows():
    xmin, ymin, xmax, ymax = map(int, [row["xmin"], row["ymin"], row["xmax"], row["ymax"]])
    rect = plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, edgecolor="lime", facecolor="none", linewidth=2)
    ax.add_patch(rect)
    ax.text(xmin, ymin - 5, f'ID {int(row["id_arbol"])}', color='yellow', fontsize=8, weight='bold')
plt.axis('off')
plt.show()

In [ ]:
# 7. Histogramas RGB
mask = np.zeros((img_array.shape[0], img_array.shape[1]), dtype=bool)
for _, row in img_out.iterrows():
    xmin, ymin, xmax, ymax = map(int, [row["xmin"], row["ymin"], row["xmax"], row["ymax"]])
    mask[ymin:ymax, xmin:xmax] = True
pixels_arbol = img_array[mask]
pixels_no_arbol = img_array[~mask]
plt.figure(figsize=(15, 5))
for i, color in enumerate(['red', 'green', 'blue']):
    plt.subplot(1, 3, i+1)
    plt.hist(pixels_arbol[:, i], bins=50, alpha=0.5, label="Árboles", color=color)
    plt.hist(pixels_no_arbol[:, i], bins=50, alpha=0.5, label="No árboles", color='gray')
    plt.title(f'Histograma canal {color.upper()}')
    plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 8. Diagramas de dispersión RGB
img_out["R_mean"] = img_out.apply(lambda row: np.mean(img_array[int(row.ymin):int(row.ymax), int(row.xmin):int(row.xmax), 0]), axis=1)
img_out["G_mean"] = img_out.apply(lambda row: np.mean(img_array[int(row.ymin):int(row.ymax), int(row.xmin):int(row.xmax), 1]), axis=1)
img_out["B_mean"] = img_out.apply(lambda row: np.mean(img_array[int(row.ymin):int(row.ymax), int(row.xmin):int(row.xmax), 2]), axis=1)
samples_out = pixels_no_arbol[np.random.choice(len(pixels_no_arbol), len(img_out), replace=False)]
R_no, G_no, B_no = samples_out[:,0], samples_out[:,1], samples_out[:,2]
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.scatter(img_out["R_mean"], img_out["G_mean"], label="Árboles", alpha=0.6)
plt.scatter(R_no, G_no, label="No árboles", alpha=0.3)
plt.title("R vs G"); plt.xlabel("R"); plt.ylabel("G"); plt.legend()
plt.subplot(1, 3, 2)
plt.scatter(img_out["G_mean"], img_out["B_mean"], label="Árboles", alpha=0.6)
plt.scatter(G_no, B_no, label="No árboles", alpha=0.3)
plt.title("G vs B"); plt.xlabel("G"); plt.ylabel("B")
plt.subplot(1, 3, 3)
plt.scatter(img_out["R_mean"], img_out["B_mean"], label="Árboles", alpha=0.6)
plt.scatter(R_no, B_no, label="No árboles", alpha=0.3)
plt.title("R vs B"); plt.xlabel("R"); plt.ylabel("B")
plt.tight_layout()
plt.show()

In [ ]:
# 9. Guardar CSV
img_out.to_csv("datos_arboles.csv", index=False)
print("Archivo 'datos_arboles.csv' guardado.")
